#Your First Homework 

In this HW, you will create transactions in Bitcoin test network. There will be exercises in the accompanying gdrive folder. You will complete and submit a completed iPython notebook to SUCourse in a zip file. 

While writing corresponding python statements, you may want to read  the related chapter(s) in Mastering Bitcoin or the resources below. 

1. Bitcoin Script: https://en.bitcoin.it/wiki/Script
2. Bitcoin Transaction Format: https://en.bitcoin.it/wiki/Transaction
3. Python-bitcoin docs: https://media.readthedocs.org/pdf/python-bitcoinlib/latest/python-bitcoinlib.pdf

Lets first install **python-bitcoinlib** which we will use to connect to the Bitcoin network and to generate a public-private key pair.

In [0]:
!pip install python-bitcoinlib
!pip install requests

    100% |████████████████████████████████| 92kB 5.8MB/s 


Lets create a Bitcoin address and accompanying private key for ourselves with the code below. 

In [0]:
from os import urandom
from bitcoin import SelectParams
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

SelectParams('testnet')

seckey = CBitcoinSecret.from_secret_bytes(urandom(32))

print("Private key: %s" % seckey)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(seckey.pub))

Private key: cV3fHjic9SRSh8iwMHMjdyz27qc37WsYEmQecV9r4ax9WaJERWF4
Address: mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod


Save this information to somewhere. Otherwise, it may dissappear if you  accidentally erase it and get disconnected. You will not be able to access it later. 

Now with this address, we can get free testnet coins from the coin faucet. Click [here](https://coinfaucet.eu/en/btc-testnet/) to get some free coins on the *Bitcoin test network*. Note that it is a nice thing to send back the bitcoin's you got to the faucet. 

Once you click the button, a transaction, which is a P2PKH transaction, will be generated and sent to the Bitcoin Test network (by the faucet). One of the outputs of this transaction will be for your Public Key Hash. That output will be our main UTXO to spend. 

Here is a sample output. Since your address is different your coins will be sent to a different address.

```
We sent 0.12133031 bitcoins to address
mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod

tx: ae2d8d7d78fb0c56582a76816a5dcdda14f945413b49ef97455b5fbf5d952be0
Send coins back, when you don't need them anymore to the address

mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB```

You can see the transaction details by checking all the transactions for your account by typing the correct address in this [web-page](https://live.blockcypher.com/btc-testnet/address/mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod/).

Before continuing for the exercises, lets configure our Python environment with our public and private key pair. This is nothing fancy; we just set some variables' values to be used later in the codes and the exercises in the rest of this notebook. 

Note that we will use the same bitcoin address, the one we created above, to correctly configure our environment. 

In [0]:
from bitcoin import SelectParams
from bitcoin.base58 import decode
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret, P2PKHBitcoinAddress

#This says that we will connect to testnet, not the actual Bitcoin network
SelectParams('testnet')

#This is my private key. You need to add yours here
my_private_key = CBitcoinSecret('cV3fHjic9SRSh8iwMHMjdyz27qc37WsYEmQecV9r4ax9WaJERWF4') 

#As you see, we do not explicitly see the public key  anywhere. We do not need to see it. 
#The elliptic curve operations are handled in the background. Your private key
#is in fact a scalar which will be multiplied with a point on the curve to obtain
#the public key (note: elliptic curve discrete logarithm problem).
my_public_key = my_private_key.pub 

#With P2PKHBitcoinAddress's from_pubkey function, we can obtain our address. 
my_address = P2PKHBitcoinAddress.from_pubkey(my_public_key)

#This is the address of the faucet (a Bitcoin Test account) which we will 
#send our money back during/after the exercises. 
faucet_address = CBitcoinAddress('mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB') 

print("Private key: %s" % my_private_key)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(my_private_key.pub))

Private key: cV3fHjic9SRSh8iwMHMjdyz27qc37WsYEmQecV9r4ax9WaJERWF4
Address: mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod


The first thing we will do will be splitting the UTXO obtained from the faucet into multiple UTXOs. There are  multiple exercises in this notebook, hence we need more than one UTXO to spend. Besides, we may do mistakes, spend an UTXO in an unlockable manner and lose some of our test coins. So it is a good exercise to split the only UTXO we have.  

Lets run the **split_coins** function below with the transaction we obtained from the faucet.  The code is given below: You do not need to change the **split_coins** function. However, you need to add the UTXO information we obtained from the faucet at the end.

You can check the status of the outputs in the original faucet transaction from BlockCypher (as I do it [here](https://live.blockcypher.com/btc-testnet/tx/1adee36659bc7268d33b3d90ebc7c7cf807f363f11a572076b3a3aa9845342d1/)). Before running the code below it must be *unspent*. Read the code and try to understand what it does by matching what we have learned in the class with the statements below.

If you do everything correctly, at the end, you need to get a response with code 201. That is, you will see something like 
```
201 Created
{
  "tx": {
        ....
  }
}
 ```
at the beginning of the output. And after some time, depending on the fee you give to the miners, the UTXO you used will appear as spent. Fortunately,  you have many more now to be used in the exercises. 

In [0]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

def create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, seckey):
    #We want no parts of the transaction to be changed.
    #Hence, we hash all the transaction before signing it (i.e.: SIGHASH_ALL). 
    sighash = SignatureHash(txin_scriptPubKey, tx, 0, SIGHASH_ALL)
    signature = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return signature

def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction)

#What we simply do in this transaction is splitting. We spent the faucet output and
#split the amount into n pieces each is again to our addresses in a P2PKH structure.
def split_coins(amount_to_send, txid_to_spend, utxo_index, n):
    #We are creating a transaction with a single input and n outputs.
    
    #txin is the input part of the transaction 
    txin_scriptPubKey = my_address.to_scriptPubKey() #initial UTXO was for my address so we need to use it as scriptPubKey 
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index)) #now we have txin
    #The function lx() onverts a little-endian hex string to bytes
    #The COutPoint is the combination of a transaction hash and an index n into its vout as we have discussed in the lecture
    
    #This transaction's outputs are all for myself - so the default P2PKH script
    #[OP_DUP OP_HASH160 <pubKeyHash> OP_EQUALVERIFY OP_CHECKSIG] must be the scriptPubKey
    #The function to_scriptPubKey automatically generates this script from a P2PKHBitcoinAddress
    #Note that the script contains a base16 encoded version of checksummed/modified byte58 encoded address (This operation is reversible!).
    txout_scriptPubKey = my_address.to_scriptPubKey() 
    
    #txout is a single output. (IMP: the function CScript which you use for the first time serializes the script created above)
    txout = CMutableTxOut((amount_to_send / n) * COIN, CScript(txout_scriptPubKey)) #amount and address are given here

    #This is the whole transaction we will broadcast
    #Note the [txout]*n in the parameters of CMutableTransaction function
    tx = CMutableTransaction([txin], [txout] * n)
    
    #Now lets sign the transaction to keep its integrity
    sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)
    #We need to create the unlocking script which is [signature, encoded SIGHASH_ALL] + PublicKey
    txin.scriptSig = CScript([sig, my_public_key])  
    
    #Lets validate if the script is correctly generated... 
    #This function raises a ValidationError subclass if the validation fails
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))
   
    #Broadcast the transaction
    response = broadcast_transaction(tx)
    print(response.status_code, response.reason)
    print(response.text)

#The mount of BTC in the output you're splitting minus fee.
#Warning: In the test network, fees are required. They can be small but since this
#is an exercise, you can be generous
amount_to_send = 0.12 

#This must be the transaction id you have from the faucet's transaction
txid_to_spend = ('ae2d8d7d78fb0c56582a76816a5dcdda14f945413b49ef97455b5fbf5d952be0')

#the output UTXO location in the transaction from faucet (mine is 1, but maybe yours is 0)
utxo_index = 1 

#number of transactions to be splitted
n = 12 
######################################################################

split_coins(amount_to_send, txid_to_spend, utxo_index, n)

Now you have learned how to send transactions to the Bitcoin network. The rest is the homework.

##Exercise 1

Here is the first exercise: it is about spending one of the smaller transactions and send it back to the **faucet**. Make it complete, run, and be sure that you have the correct output with code *201 Created*.

**Hint**:  Almost all the information you need, the codes, functions to be used etc. can be read and copied from the **split_coins** example above. Before, we created a transaction to split the original UTXO obtained from the faucet. You will do the same here but this time you will not direct the output to yourself. The money needs to go back to the faucet.

In [0]:
from bitcoin.core.script import *

# TODO: Complete this script to unlock the BTC that was sent to you
# in the PayToPublicKeyHash transaction. You may need to use variables
# that are globally defined.
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = #todo
    txout = #todo
    tx = CMutableTransaction([txin], [txout])

    txin_scriptPubKey = #todo
    signature = #todo
    txin.scriptSig = #todo
    
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

    return broadcast_transaction(tx)
  
######################################################################
# TODO: set all these parameters correctly
amount_to_send =  #todo  
txid_to_spend = #todo
utxo_index = #todo
txout_scriptPubKey = #todo
######################################################################

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

###You need to complete Exercise 1 before going for Exercise 2 and Exercise 3.

#Exercise 2: 

In this exercise, you will create a UTXO and later redeem it by using P2SH approach. The locking script will contain the hash of a puzzle whose solution will be given in the redeem script. Let **a** be the number of letters in your firstname and let **b** be the number of letters in your lastname. 

To unlock, the locking script will require two integer values whose difference is  **a - b** and sum is **a + b**. Once the redeemer presents **a** and **b** in the unlocking script (it will contain only **a** and **b**) as well as the redemption script itself, he will be able to unlock the UTXO. 

For the opcodes you may need to use please check this [webpage](https://en.bitcoin.it/wiki/Script). Again if you are successful, you will get the return code **201** after the transaction broadcast.

You will complete this exercise in two parts. First a transaction will be created with a single output. Then it will be redeemed (by youself, to be sent to the faucet). The starting code for the first part is given below:



In [0]:
from sys import exit
from bitcoin.core.script import *

######################################################################
# TODO: Complete the scriptPubKey implementation with operations 
# Be careful while using OP_EQUAL and OP_VERIFY - read them from https://en.bitcoin.it/wiki/Script
ex2a_txout_scriptPubKey = #todo
######################################################################

######################################################################
# TODO: set these parameters correctly
amount_to_send = #todo
txid_to_spend = #todo
utxo_index = #todo
######################################################################

#This function is from the first exercise
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, ex2a_txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

In the second part, we will redeem by only providing the solution of the locking script. Note that anyone can do this. Since the lock script does not contain any signature, address etc. check, one only need to provide **a** and **b**. You can follow the results, e.g., confirmations, from BlockCypher test network explorer by using the transaction hash you obtained after succesfully completing this part. 

In [0]:
def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = #todo
    txin = #todo
    tx = #todo

    txin.scriptSig = #todo
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    
    return broadcast_transaction(tx)

######################################################################
# TODO: set these parameters correctly
amount_to_send = #todo
txid_to_spend = #todo
utxo_index = #todo
######################################################################

txin_scriptPubKey = ex2a_txout_scriptPubKey
######################################################################
# TODO: implement the scriptSig for redeeming the transaction created in  Exercise 2a.
# This must be a very chort scriptSig. No signature, address check etc. 
# Anyone who can submit the solution of the problem (a and b in correct form) can spend the transaction
txin_scriptSig = #todo
######################################################################
txout_scriptPubKey = #todo #sending money to faucet after solving the puzzle

response = send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

##Exercise 3:

In the first part of the third exercise, you will create and broadcast a MULTISIG transaction to the test network. The second part will be about providing an unlock script for this transaction. You will definitely want to read how OP_CHECKMULTISIG works from [here](https://en.bitcoin.it/wiki/Script).

The scenario is as follows: in a company, there is a manager and three employees. The multisig signature 
you will create (in the first part) can be redeemed by the manager combined with one of the employees. However, only the manager, or only the employees with enough number of signatures will not be able to redeem the transaction. 

You may assume the role of the manager for this exercise so that the manager’s private key is your private key and the manager’s public key is your public key. 

For the employee private keys and addresses, you can use the key generation code at the beginning of this notebook. That is generate 3 random Bitcoin secret keys to be used in this exercise with the code below. 


In [0]:
from os import urandom
from bitcoin import SelectParams
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

SelectParams('testnet')

seckey = CBitcoinSecret.from_secret_bytes(urandom(32))

print("Private key: %s" % seckey)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(seckey.pub))

After filling the spaces ---xxx-- in the first part below, we can create the transaction locked as described above. 

In [0]:
from sys import exit
from bitcoin.core.script import *

#Run the key generation code at the beginning of this notebook multiple times 
#to create private keys and addresses for the employees. 
employee1_private_key = #todo
employee1_public_key = #todo
employee2_private_key = #todo
employee2_public_key = #todo
employee3_private_key = #todo
employee3_public_key = #todo

######################################################################
# TODO: Complete the scriptPubKey implementation for Exercise 3
# You can assume the role of the manager for the purposes of this problem
# and use my_public_key and my_private_key in lieu of manager_public_key and
# manager_private_key.

ex3a_txout_scriptPubKey = #todo

######################################################################

######################################################################
# TODO: set these parameters correctly
amount_to_send = #todo
txid_to_spend = #todo
utxo_index = #todo
######################################################################

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, ex3a_txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)


In the second part, you will redeem the transaction. Keep scriptPubKey  as small as you can. One can use any legal combination of signatures to redeem the transaction but make sure that all combinations would have worked. Please note the order of the public keys (in the locking script) and signatures (in the unlocking script). 

In [0]:
from sys import exit
from bitcoin.core.script import *

#employee ID is 1,2 or 3
def multisig_scriptSig(tx, txin_scriptPubKey, employee_ID):
    manager_sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)
    if employee_ID == 1:
      employee_sig = #todo
    elif employee_ID == 2:
      employee_sig = #todo
    else:
      employee_sig = #todo
    ######################################################################
    # TODO: Complete this script to unlock the BTC that was locked in the multisig transaction created in Exercise 3a.
    return #todo
    ######################################################################

def send_from_multisig_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txout_scriptPubKey):
    txin = #todo
    txout = #todo
    tx = #todo

    txin.scriptSig = #todo
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    return broadcast_transaction(tx)

######################################################################
# TODO: set these parameters correctly
amount_to_send = #todo
txid_to_spend = #todo
utxo_index = #todo
######################################################################

txin_scriptPubKey = ex3a_txout_scriptPubKey
txout_scriptPubKey = faucet_address.to_scriptPubKey()

response = send_from_multisig_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

###Once you are done, submit your complete iPython notebook (.ipynb) to SUCourse by changing the file name with your_name_surname.ipynb. 